In [ ]:
!pip install yfinance

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import regression as reg
from scipy import stats
from matplotlib import pyplot as plt
import pickle
import netpixi
from netpixi.integration.gt import *
from regression.integration.gt import *
from graph_tool import centrality, clustering, spectral
import cpnet
from scipy.spatial.distance import pdist, squareform

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
df = pd.read_csv('etf_funds/ETFs.csv')

In [3]:
fund_names = df["fund_symbol"].values

In [4]:
funds_historical_data = yf.download(list(fund_names), period="5y")

[                       0%                       ]  7 of 2310 completed

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf838c040>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d841b7340>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d13049ea0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cd87ed960>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d13049780>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf838c0a0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d118c2980>


[                       1%                       ]  15 of 2310 completed

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf814a320>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d1304a9e0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d118c2b00>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf84b1c60>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d118c2980>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf84b1c00>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d118c28c0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d841b7340>


[                       1%                       ]  20 of 2310 completed

...

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf833bfa0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cb7d60>


[*********************100%***********************]  2307 of 2310 completed

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cc9c00>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cb7ee0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cb7d60>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6cf829f400>


[*********************100%***********************]  2310 of 2310 completed

2 Failed downloads:
- MJJ: No data found, symbol may be delisted
- REML: No data found, symbol may be delisted


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cde5c0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6d12cb6c80>


In [5]:
funds_historical_data["Adj Close"].to_csv("etf_funds/historical_data_new_5y.csv")
funds_historical_data["Volume"].to_csv("etf_funds/volume_new_5y.csv")

In [6]:
file_df = pd.read_csv("etf_funds/historical_data_new_5y.csv")
volume_df = pd.read_csv("etf_funds/volume_new_5y.csv")

In [7]:
fund_names = list(file_df.columns[1:].values)
fund_returns = []
fund_vol  = []
fund_sharpe = []
fund_volume = []


for fund in fund_names:
    fund_list = file_df[fund].values
    fund_return = (fund_list[-1] - fund_list[0]) / fund_list[0]
    fund_volatility = file_df[fund].pct_change().std() * np.sqrt(252)
    fund_mean_daily_volume = volume_df[fund].mean()
    
    fund_returns.append(fund_return)
    fund_vol.append(fund_volatility)
    fund_sharpe.append(fund_return / fund_volatility)
    fund_volume.append(fund_mean_daily_volume)
 

In [8]:
df_final = pd.DataFrame(data={ 
    "Symbol": fund_names, 
    "Return": fund_returns, 
    "Volatility": fund_vol, 
    "Sharpe": fund_sharpe,
    "Volume": fund_volume
})

In [9]:
# Removendo fundos que já encerraram as operações
df_final = df_final.dropna()

In [10]:
df_final

,Symbol,Return,Volatility,Sharpe,Volume
2,AADR,-0.138521,0.244183,-0.567284,1.431660e+04
3,AAXJ,-0.113130,0.227159,-0.498020,1.246921e+06
7,ACSI,0.556156,0.206869,2.688447,1.581096e+03
10,ACWF,0.181302,0.202850,0.893772,2.369555e+04
11,ACWI,0.359218,0.202702,1.772148,3.582106e+06
...,...,...,...,...,...
2303,YXI,0.017341,0.293103,0.059164,1.373058e+04
2304,YYY,-0.119523,0.170868,-0.699505,1.168822e+05
2307,ZIVZF,-0.687064,0.437079,-1.571945,4.897133e+04
2308,ZROZ,-0.203576,0.225650,-0.902173,7.098832e+04


In [24]:
# Recorte Volume
corte_volume = df_final["Volume"].quantile(0.2)

df_recorte_volume = df_final[df_final["Volume"] > corte_volume]

In [25]:
# Recorte Sharpe
corte_sharpe_acima = df_final["Sharpe"].quantile(0.15)
corte_sharpe_abaixo = df_final["Sharpe"].quantile(0.85)

df_recorte_sharpe = df_final[
    (df_final["Sharpe"] > corte_sharpe_acima) & \
    (df_final["Sharpe"] < corte_sharpe_abaixo)
]

In [26]:
# Recorte Sharpe-Volume
df_recorte_sharpe_volume = df_final[
    (df_final["Volume"] > corte_volume) & \
    (df_final["Sharpe"] > corte_sharpe_acima) & \
    (df_final["Sharpe"] < corte_sharpe_abaixo)
]

In [ ]:
# fund_holdings = {}
# unique_assets = []
# error_list = []

# for index, fund in enumerate(list(df_final["Symbol"].values)):
#     try:
#         asset = yf.Ticker(fund)
#         holdings_info = asset.get_info()['holdings']
#         holdings = {}
        
#         for holding in holdings_info:
#             holdings[holding["holdingName"]] = holding["holdingPercent"]
#             unique_assets.append(holding["holdingName"])
        
#         fund_holdings[fund] = holdings
#         if (index % 100 == 0):
#             print(index)
#     except:
#         error_list.append(fund)

In [ ]:
# f = open('fund_holdings.pckl', 'wb')
# pickle.dump(fund_holdings, f)
# f.close()

# f = open('unique_assets.pckl', 'wb')
# pickle.dump(list(set(unique_assets)), f)
# f.close()

# f = open('error_list.pckl', 'wb')
# pickle.dump(error_list, f)
# f.close()

In [14]:
f = open('error_list.pckl', 'rb')
error_list = pickle.load(f)
f.close()

f = open('unique_assets.pckl', 'rb')
unique_assets = pickle.load(f)
f.close()

f = open('fund_holdings.pckl', 'rb')
fund_holdings = pickle.load(f)
f.close()

In [15]:
def gera_csv(error_list, unique_assets, fund_holdings, df, nome_arquivo):
    matrix = pd.DataFrame(columns=list(set(unique_assets)), index=fund_holdings.keys())
    matrix.fillna(0, inplace=True)

    invalid_funds = []

    for fund in fund_holdings:
        for asset in fund_holdings[fund]:
            if fund_holdings[fund][asset] > 1:
                invalid_funds.append(fund)
            matrix.loc[fund, asset] = fund_holdings[fund][asset]
    
    matrix = matrix.drop(list(set(invalid_funds)))
    
    distances = pd.DataFrame(squareform(pdist(matrix, 'euclidean')), columns=matrix.index, index=matrix.index)
    
    media_jrg = distances.mean().mean() - 0.8 * distances.std().mean()
    
    g = Graph(directed=False) # não-dirigido
    
    for fund in fund_holdings.keys():
        try:
            g.add_vertex(fund)
        except:
            print(fund)
    
    for row, col in distances.iterrows():
        for index, value in col.items():
            if row == index: 
                continue

            if value > media_jrg:
                continue

            try:
                g.add_edge(row, index)
            except:
                pass
    
    g = gt_clean(g)
    
    bc, _ = centrality.betweenness(g)
    g.add_vp('betweenness', bc)
    gt_data(g).loc[:, :].sort_values('betweenness', ascending=False)
    
    matrix = spectral.adjacency(g)

    algorithm = cpnet.Rossa()
    algorithm.detect(matrix)
    c = algorithm.get_coreness()

    g.add_vp('coreness')
    for i, coreness in c.items():
        v = g.get_vertex_by_index(i)
        v['coreness'] = float(coreness)
    
    lc = clustering.local_clustering(g)
    g.add_vp('clustering', lc)
    
    ntw_metrics = gt_data(g)
    
    metrics = pd.merge(df[['Symbol','Return', 'Volatility', 'Sharpe', 'Volume']], ntw_metrics, left_on='Symbol', right_on='id', how='inner')
    
    metrics.to_csv(f"{nome_arquivo}.csv")

In [16]:
gera_csv(error_list, unique_assets, fund_holdings, df_final, "metrics_new")

In [28]:
gera_csv(error_list, unique_assets, fund_holdings, df_recorte_sharpe, "metrics_sharpe")

In [29]:
gera_csv(error_list, unique_assets, fund_holdings, df_recorte_volume, "metrics_volume")

In [27]:
gera_csv(error_list, unique_assets, fund_holdings, df_recorte_sharpe_volume, "metrics_sharpe_volume")

In [18]:
matrix = pd.DataFrame(columns=list(set(unique_assets)), index=fund_holdings.keys())
matrix.fillna(0, inplace=True)

invalid_funds = []

for fund in fund_holdings:
    for asset in fund_holdings[fund]:
        if fund_holdings[fund][asset] > 1:
            invalid_funds.append(fund)
        matrix.loc[fund, asset] = fund_holdings[fund][asset]

In [19]:
matrix = matrix.drop(list(set(invalid_funds)))

In [20]:

distances = pd.DataFrame(squareform(pdist(matrix, 'euclidean')), columns=matrix.index, index=matrix.index)

distances

,AAA,AAAU,AADR,AAXJ,ABEQ,ACES,ACIO,ACSI,ACTV,ACVF,...,YLDE,YMAR,YOLO,YPS,YXI,YYY,ZHDG,ZIG,ZROZ,ZSL
AAA,0.000000,1.012001,0.205556,0.195133,0.230977,0.223306,0.203392,0.229159,0.181673,0.177798,...,0.189402,0.971054,0.390675,0.157000,0.479807,0.208836,0.155390,0.219455,0.163359,0.972715
AAAU,1.012001,0.000000,1.009013,1.006941,1.014497,1.012778,1.008574,1.014085,1.004420,1.003726,...,1.005846,1.385208,1.062300,1.000251,1.098211,1.009686,1.000000,1.011936,1.001269,1.386372
AADR,0.205556,1.009013,0.000000,0.178992,0.217512,0.209348,0.187961,0.215580,0.164214,0.159917,...,0.172726,0.967940,0.382868,0.136419,0.473472,0.193839,0.134563,0.205236,0.143692,0.969606
AAXJ,0.195133,1.006941,0.178992,0.000000,0.207690,0.199124,0.176502,0.205666,0.150963,0.146277,...,0.160180,0.965780,0.377375,0.120141,0.469041,0.182749,0.118029,0.194796,0.128340,0.967450
ABEQ,0.230977,1.014497,0.217512,0.207690,0.000000,0.234358,0.215468,0.239942,0.195098,0.181068,...,0.202315,0.973655,0.397096,0.172358,0.485049,0.220615,0.170893,0.230692,0.178170,0.975311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YYY,0.208836,1.009686,0.193839,0.182749,0.220615,0.212569,0.191543,0.218710,0.168301,0.164111,...,0.176616,0.968642,0.384639,0.141312,0.474905,0.000000,0.139522,0.208521,0.148346,0.970306
ZHDG,0.155390,1.000000,0.134563,0.118029,0.170893,0.160372,0.131233,0.168427,0.094122,0.086406,...,0.108290,0.958541,0.358442,0.022424,0.453948,0.139522,0.000000,0.154966,0.050400,0.960223
ZIG,0.219455,1.011936,0.205236,0.194796,0.230692,0.223011,0.203068,0.228871,0.181310,0.177427,...,0.189054,0.970987,0.390507,0.156580,0.479670,0.208521,0.154966,0.000000,0.162956,0.972647
ZROZ,0.163359,1.001269,0.143692,0.128340,0.178170,0.168105,0.140579,0.175806,0.106767,0.100031,...,0.119444,0.959865,0.361968,0.055163,0.456737,0.148346,0.050400,0.162956,0.000000,0.961545


In [37]:
media_jrg = distances.mean().mean() - 0.8 * distances.std().mean()

(0.1354992594336807, 0.2948676563239358, 0.17872214018914204)

In [ ]:
g = Graph(directed=False) # não-dirigido

In [ ]:
for fund in fund_holdings.keys():
    try:
        g.add_vertex(fund)
    except:
        print(fund)

In [ ]:
for row, col in distances.iterrows():
    for index, value in col.items():
        if row == index: 
            continue
            
        if value > media_jrg:
            continue
        
        try:
            g.add_edge(row, index)
        except:
            pass

In [ ]:
n = g.num_vertices()
n

In [ ]:
m = g.num_edges()
m

In [ ]:
g = gt_clean(g)

In [ ]:
bc, _ = centrality.betweenness(g)
g.add_vp('betweenness', bc)
gt_data(g).loc[:, :].sort_values('betweenness', ascending=False)

In [ ]:
matrix = spectral.adjacency(g)

algorithm = cpnet.Rossa()
algorithm.detect(matrix)
c = algorithm.get_coreness()

g.add_vp('coreness')
for i, coreness in c.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)

In [ ]:
lc = clustering.local_clustering(g)
g.add_vp('clustering', lc)

In [ ]:
ntw_metrics = gt_data(g)

In [ ]:
metrics = pd.merge(df_final[['Symbol','5Y Return', '5Y Volatility', '5Y Sharpe']], ntw_metrics, left_on='Symbol', right_on='id', how='inner')

In [ ]:
metrics["Antifragile"] = 1 / metrics["5Y Volatility"]

In [ ]:
metrics

In [ ]:
metrics.to_csv("metricas_5y.csv")